# Исследование опроса клиентов телекомунникацонной компании

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CAST (CASE
                 WHEN u.lt_day <= 365 THEN 1
                 WHEN u.lt_day > 365 THEN 0
             END AS boolean) AS is_new,
       u.age,
       CASE
           WHEN u.gender_segment IS NOT NULL AND u.gender_segment = 1 THEN 'женщина'
           WHEN u.gender_segment IS NOT NULL AND u.gender_segment = 0 THEN 'мужчина'
       END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       u.nps_score,
       l.country,
       l.city,
       ags.title AS age_segment,
       ts.title AS traffic_segment,
       lts.title AS lifetime_segment,
       CASE
           WHEN u.nps_score = 10 OR u.nps_score = 9 THEN "cторонники"
           WHEN u.nps_score = 8 OR u.nps_score = 7 THEN "нейтралы"
           WHEN u.nps_score < 7 THEN "критики"
       END AS nps_group
FROM user AS u
LEFT JOIN location AS l ON u.location_id = l.location_id
LEFT JOIN age_segment AS ags ON u.age_gr_id = ags.age_gr_id
LEFT JOIN traffic_segment AS ts ON u.tr_gr_id = ts.tr_gr_id
LEFT JOIN lifetime_segment AS lts ON u.lt_gr_id = lts.lt_gr_id;
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,nps_score,country,city,age_segment,traffic_segment,lifetime_segment,nps_group
0,A001A2,2320,0,45.0,женщина,ANDROID,SMARTPHONE,10,Россия,Уфа,05 45-54,04 1-5,08 36+,cторонники
1,A001WF,2344,0,53.0,мужчина,ANDROID,SMARTPHONE,10,Россия,Киров,05 45-54,04 1-5,08 36+,cторонники
2,A003Q7,467,0,57.0,мужчина,ANDROID,SMARTPHONE,10,Россия,Москва,06 55-64,08 20-25,06 13-24,cторонники
3,A004TB,4190,0,44.0,женщина,IOS,SMARTPHONE,10,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,cторонники
4,A004XT,1163,0,24.0,мужчина,ANDROID,SMARTPHONE,10,Россия,Рязань,02 16-24,05 5-10,08 36+,cторонники
5,A005O0,5501,0,42.0,женщина,ANDROID,SMARTPHONE,6,Россия,Омск,04 35-44,05 5-10,08 36+,критики
6,A0061R,1236,0,45.0,мужчина,ANDROID,SMARTPHONE,10,Россия,Уфа,05 45-54,06 10-15,08 36+,cторонники
7,A009KS,313,1,35.0,мужчина,ANDROID,SMARTPHONE,10,Россия,Москва,04 35-44,13 45-50,05 7-12,cторонники
8,A00AES,3238,0,36.0,женщина,ANDROID,SMARTPHONE,10,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,cторонники
9,A00F70,4479,0,54.0,женщина,ANDROID,SMARTPHONE,9,Россия,Волгоград,05 45-54,07 15-20,08 36+,cторонники


In [5]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

<div class="alert alert-info">  
    
[Презентация](https://disk.yandex.ru/d/yX4Vvv4XB6XL3A)
    
[Дашборд](https://public.tableau.com/shared/CH5Y6PS9Q?:display_count=n&:origin=viz_share_link)
</div>